In [13]:
import seaborn
import matplotlib
import torch
import numpy as np
import pandas as pd
import yfinance as yf
import torch.nn as nn
import os
import time
import datetime
import pathlib
from torch.utils.data import DataLoader
import kan


In [14]:
data_names = ["S&P500","SSE","IBM","MSFT","PAICC"]
data_name = data_names[0]
data_path="./datasets/S&P500.csv"
print(data_path)
dataframe = pd.read_csv(data_path)
dataframe.describe()
print("shape = ",dataframe.shape)

./datasets/S&P500.csv
shape =  (5031, 7)


In [15]:
def add_Ma(dataframe):
  Ma_window=5
  for i in range(0,dataframe.shape[0]-Ma_window):
    dataframe.loc[dataframe.index[i+Ma_window],'Ma'] = np.round(((dataframe.iloc[i,4]+ dataframe.iloc[i+1,4] +dataframe.iloc[i+2,4] + dataframe.iloc[i+3,4]+ dataframe.iloc[i+4,4])/5),6)
  return dataframe[5:-5]

dataframe=add_Ma(dataframe)
dataframe.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ma
5,1999-01-11,1275.089966,1276.219971,1253.339966,1263.880005,1263.880005,818000000,1258.007983
6,1999-01-12,1263.880005,1264.449951,1238.290039,1239.510010,1239.510010,800200000,1265.163989
7,1999-01-13,1239.510010,1247.750000,1205.459961,1234.400024,1234.400024,931500000,1264.109985
8,1999-01-14,1234.400024,1236.810059,1209.540039,1212.189941,1212.189941,797200000,1256.521997
9,1999-01-15,1212.189941,1243.260010,1212.189941,1243.260010,1243.260010,798100000,1245.013989


In [16]:
import torch
from torch import nn

class GeneratorModel(nn.Module):
    def __init__(self, n_sequence, n_features):
        super(GeneratorModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=n_features, hidden_size=10, batch_first=True)
        self.batch_norm1 = nn.BatchNorm1d(10)
        self.leaky_relu = nn.LeakyReLU(0.3)
        self.dropout1 = nn.Dropout(0.3)
        
        self.lstm2 = nn.LSTM(input_size=10, hidden_size=10, batch_first=True)
        self.batch_norm2 = nn.BatchNorm1d(10)  
        self.dropout2 = nn.Dropout(0.3)
        
        self.output_dense = nn.Linear(10, n_features)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.batch_norm1(x.permute(0, 2, 1)).permute(0, 2, 1) 
        x = self.leaky_relu(x)
        x = self.dropout1(x)

        _, (x, _) = self.lstm2(x)
        x=x.permute(1, 0, 2)
        x = self.batch_norm2(x.permute(0, 2, 1)).permute(0, 2, 1)  
        x = self.leaky_relu(x)
        x = self.dropout2(x)

        x = self.output_dense(x)
        x = self.leaky_relu(x)

        return x

# Example usage
n_sequence = 5  # Sequence length
n_features = 7   # Number of features


class KAN_discriminator(torch.nn.Module):
    def __init__(self, n_sequence, n_features):
        super(KAN_discriminator, self).__init__()
        input_dim = (n_sequence + 1) * n_features
        
        self.model = nn.Sequential(
            nn.Flatten(),
            kan.KAN([42,64,10,1]),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [17]:
torch.set_printoptions(precision=8)
class Standarized_TimeseriesGenerator(torch.utils.data.Dataset):
    def __init__(self, data, targets, length, batch_size=1, stride=1):
        self.data = data
        self.targets = targets
        self.length = length
        self.batch_size = batch_size
        self.stride = stride
        
    def __getitem__(self, index):
        samples = [self.data[i:i+self.length] for i in range(index, len(self.data)-self.length, self.stride)]
        targets = [self.targets[i+self.length] for i in range(index, len(self.data)-self.length, self.stride)]
        # Pack sequences into tensor
        samples = torch.tensor(samples[0])
        targets = torch.tensor(targets[0])
        samples= samples.to(torch.float64)
        targets= targets.to(torch.float64)
        # shape : (n_batch, n_sequence, n_features)
        mean = samples.mean(dim=0)
        std = samples.std(dim=0,correction=0)
        samples = (samples - mean)/std  # standardize along each feature


        # targets = (targets - mean[..., 3])/std[..., 3]  # The close value is our target
        targets = (targets - mean)/std  # The close value is our target
        return samples, targets
    
    def __len__(self):
        return len(self.data) - self.length


In [18]:
n_sequence = 5
n_features = 7
n_batch = 32

def get_gen_train_test(dataframe):
  data = dataframe.drop(columns='Date').to_numpy()
  #targets = data[:,3, None] #add none to have same number of dimensions as data
  targets = data
  n_samples = data.shape[0]
  train_test_split=int(n_samples*0.9)
  train_data = data[:train_test_split]
  test_data = data[train_test_split:]
  train_target = targets[:train_test_split]
  test_target = targets[train_test_split:]
  data_train = Standarized_TimeseriesGenerator(train_data, train_target,
                                length=n_sequence, 
                                stride=1)
  data_test = Standarized_TimeseriesGenerator(test_data, test_target,
                                length=n_sequence, 
                                stride=1)
  
  train_loader = DataLoader(data_train, batch_size=n_batch, shuffle=True)
  test_loader = DataLoader(data_test, batch_size=n_batch, shuffle=False)
  return train_loader, test_loader

data_gen_train, data_gen_test = get_gen_train_test(dataframe)

In [19]:
import matplotlib.pyplot as plt
import torch
from torchvision.utils import make_grid
import tqdm
import time

torch.manual_seed(8)

def train_GAN(G, D, optim_G, optim_D, loss_f, train_loader, test_loader, num_epochs, device):
    test_size = len(test_loader)
    best = np.inf 
    for epoch in range(num_epochs):
        for i,data in enumerate(train_loader):
            if i<140:
                generator.train()
                discriminator.train()   
                starting_time = time.time()
                sequence , target = data
                sequence = sequence.to(device)
                target = target.to(device)
                generator_fake = G(sequence)
                true_seq = torch.concat((sequence,target.reshape(-1,1,7)),dim=1)
                fake_seq = torch.concat((sequence,generator_fake),dim=1)
                # ========================
                #   Train Discriminator
                # ========================
                # train with real data
                
                prediction = D(true_seq)

                # train with fake data
                
                fake_predection = D(fake_seq)
                d_loss = descriminator_loss(prediction, fake_predection)
                # update D
                
                D.zero_grad()
                d_loss.backward()
                optim_D.step()

                # ========================
                #   Train Generator
                # ========================
                # train with fake data  
                        
                sequence , target = data
                sequence = sequence.to(device)
                target = target.to(device)
                generator_fake = G(sequence)
                fake_seq = torch.concat((sequence,generator_fake),dim=1)
                fake_predection = D(fake_seq)
                g_loss, mse_loss = generator_loss(generator_fake,target,fake_predection)
                # update G
                G.zero_grad()
                g_loss.backward()
                optim_G.step()   
        print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f \tMSE_loss: %.4f \tTime: %.4f'% (epoch, num_epochs, i, len(train_loader), d_loss.item(), g_loss.item(),mse_loss.item(),time.time()-starting_time))
        #evaluate 
        dis_loss = 0
        gen_loss= 0
        mse_loss = 0
        diss_losses = []
        gen_losses = []
        mse_losses = []

        for i,data in enumerate(test_loader):
            generator.eval()
            discriminator.eval()
            with torch.no_grad():
                sequence , target = data
                sequence = sequence.to(device)
                target = target.to(device)
                generator_fake = G(sequence)
                fake_seq = torch.concat((sequence,generator_fake),dim=1)
                true_seq = torch.concat((sequence,target.reshape(-1,1,7)),dim=1)
                fake_predection = D(fake_seq)
                g_loss, mse = generator_loss(generator_fake,target,fake_predection)
                prediction = D(true_seq)
                d_loss = descriminator_loss(prediction, fake_predection)
                dis_loss += d_loss.item()/test_size
                gen_loss += g_loss.item()/test_size
                mse_loss += mse.item()/test_size
        diss_losses.append(dis_loss)
        gen_losses.append(gen_loss)
        mse_losses.append(mse_loss)

        if mse_loss < best:
            best = mse_loss
            torch.save({
                'model_state_dict': G.state_dict(),
                'optimizer_state_dict': optim_G.state_dict(),
            }, 'Kan_gan.pth') 
        print('Validation \tLoss_D: %.4f\tLoss_G: %.4f \tMSE_loss: %.4f \tBest_loss: %.4f'% (dis_loss, gen_loss,mse_loss,best))
    return diss_losses, gen_losses, mse_losses


    
loss_fn = nn.BCELoss()
mse_fn = nn.MSELoss()

a1 = 0.01
def descriminator_loss(real_output, fake_output):
    real_loss = loss_fn(real_output, torch.ones_like(real_output))
    fake_loss = loss_fn(fake_output, torch.zeros_like(fake_output))
    return real_loss + fake_loss
def generator_loss(x,y,fake_output):
    loss = loss_fn(fake_output, torch.ones_like(fake_output))
    mse_loss = mse_fn(x,y.reshape(-1,1,7))
    return a1*mse_loss + (1-a1)*loss , mse_loss

epochs = 200
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = GeneratorModel(n_sequence, n_features).to(torch.float64)
discriminator = KAN_discriminator(n_sequence,n_features).to(torch.float64)
generator , discriminator = generator.to(device), discriminator.to(device)

learning_rate1 = 0.00003
learning_rate2 = 0.00003
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate1, betas=(0.5, 0.999))
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate2, betas=(0.5, 0.999))
checkpoint_path = "./train_checkpoints"

result = train_GAN(generator, discriminator, generator_optimizer, discriminator_optimizer, loss_fn, data_gen_train, data_gen_test, epochs, device)

42 64
64 10
finished
[0/200][141/142]	Loss_D: 1.3993	Loss_G: 0.7698 	MSE_loss: 5.9770 	Time: 0.0556
Validation 	Loss_D: 1.3698	Loss_G: 0.7590 	MSE_loss: 5.5409 	Best_loss: 5.5409
[1/200][141/142]	Loss_D: 1.3666	Loss_G: 0.7458 	MSE_loss: 4.6481 	Time: 0.0573
Validation 	Loss_D: 1.3427	Loss_G: 0.7625 	MSE_loss: 5.5018 	Best_loss: 5.5018
[2/200][141/142]	Loss_D: 1.3489	Loss_G: 0.7602 	MSE_loss: 5.0756 	Time: 0.0516
Validation 	Loss_D: 1.3155	Loss_G: 0.7702 	MSE_loss: 5.4310 	Best_loss: 5.4310
[3/200][141/142]	Loss_D: 1.2972	Loss_G: 0.7553 	MSE_loss: 3.8531 	Time: 0.0626
Validation 	Loss_D: 1.2858	Loss_G: 0.7819 	MSE_loss: 5.3966 	Best_loss: 5.3966
[4/200][141/142]	Loss_D: 1.2488	Loss_G: 0.7773 	MSE_loss: 4.4865 	Time: 0.0552
Validation 	Loss_D: 1.2555	Loss_G: 0.7962 	MSE_loss: 5.3472 	Best_loss: 5.3472
[5/200][141/142]	Loss_D: 1.3231	Loss_G: 0.7750 	MSE_loss: 4.6449 	Time: 0.0571
Validation 	Loss_D: 1.2283	Loss_G: 0.8068 	MSE_loss: 5.2460 	Best_loss: 5.2460
[6/200][141/142]	Loss_D: 1.2587

In [20]:
result = train_GAN(generator, discriminator, generator_optimizer, discriminator_optimizer, loss_fn, data_gen_train, data_gen_test, epochs, device)

[0/200][141/142]	Loss_D: 0.9106	Loss_G: 1.8722 	MSE_loss: 4.5771 	Time: 0.0622
Validation 	Loss_D: 1.0385	Loss_G: 1.4600 	MSE_loss: 3.7179 	Best_loss: 3.7179
[1/200][141/142]	Loss_D: 1.2289	Loss_G: 1.7469 	MSE_loss: 4.5665 	Time: 0.0575
Validation 	Loss_D: 1.0382	Loss_G: 1.4561 	MSE_loss: 3.7294 	Best_loss: 3.7179
[2/200][141/142]	Loss_D: 0.8072	Loss_G: 2.0602 	MSE_loss: 4.0434 	Time: 0.0592
Validation 	Loss_D: 1.0506	Loss_G: 1.4207 	MSE_loss: 3.7315 	Best_loss: 3.7179
[3/200][141/142]	Loss_D: 0.7633	Loss_G: 1.6446 	MSE_loss: 6.3960 	Time: 0.0607
Validation 	Loss_D: 1.0053	Loss_G: 1.5097 	MSE_loss: 3.7797 	Best_loss: 3.7179
[4/200][141/142]	Loss_D: 0.9397	Loss_G: 1.6017 	MSE_loss: 3.3865 	Time: 0.0496
Validation 	Loss_D: 1.0393	Loss_G: 1.4185 	MSE_loss: 3.7604 	Best_loss: 3.7179
[5/200][141/142]	Loss_D: 0.9730	Loss_G: 1.2528 	MSE_loss: 2.9248 	Time: 0.0505
Validation 	Loss_D: 1.0189	Loss_G: 1.4819 	MSE_loss: 3.7388 	Best_loss: 3.7179
[6/200][141/142]	Loss_D: 1.0749	Loss_G: 1.6898 	MSE_